<a href="https://colab.research.google.com/github/ct-williams/TCBLNet/blob/main/CS230_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import numpy as np
import h5py

In [3]:
# Load Data
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# Deep Learning Approach

  # Standard Layer Definition
class StandardLayer(nn.Module):
  def __init__(self, n_input_features, n_output_features):
    super(StandardLayer, self).__init__()
    self.lin = nn.Linear(n_input_features, n_output_features)
    self.LRelu = nn.LeakyReLU(0.01)

  def forward(self, x):
    y_hat = self.LRelu(self.lin(x))
    return y_hat

# Model Definition
Model = nn.Sequential(StandardLayer(240,64), # Layer 1
                      StandardLayer(64,32),  # Layer 2
                      StandardLayer(32,32),  # Layer 2
                      StandardLayer(32,16),  # Layer 3
                      StandardLayer(16,16),  # Layer 4
                      StandardLayer(16,8),   # Layer 5
                      StandardLayer(8,8),    # Layer 6
                      nn.Linear(8, 2))       # Layer 7 - Output Layer
Model.double()

# Loss Function 
LossFunction = nn.MSELoss()

# Optimizer 
Optimizer = torch.optim.Adam(Model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.00)

In [5]:
# Training
num_epochs = 700
num_batches = 180
train_dir = "/content/drive/MyDrive/Colab Notebooks/training_data/train/"

for epoch in range(num_epochs):
  with torch.no_grad():
     norm_error = 0
  for batch in range(1, num_batches+1): 
     # Load data from batch (i) 
     filename = train_dir+"batch_"+ str(batch).zfill(5) + ".hdf"
     f = h5py.File(filename, 'r')
     data = np.transpose(f['data'])
     Y = data[:,0:2]
     X = data[:,2:]
     # Convert numpy arrays into tensors
     inputs = torch.from_numpy(X)
     labels = torch.from_numpy(Y)
     # Forward propagation
     predictions = Model(inputs)
     cost = LossFunction(predictions, labels)
     # Backward propagation
     Optimizer.zero_grad()
     cost.backward()
     Optimizer.step()
     with torch.no_grad():
       norm_error += cost.item()
  if (epoch)%50 == 0:
     print('epoch = ',{epoch},'cost = ', {norm_error/(num_batches)})

epoch =  {0} cost =  {1099.9033682935615}
epoch =  {50} cost =  {9.283237029143887}
epoch =  {100} cost =  {8.195713338555537}
epoch =  {150} cost =  {7.375476300758432}
epoch =  {200} cost =  {6.7086833192371085}
epoch =  {250} cost =  {6.1560239702626}
epoch =  {300} cost =  {5.817284050632982}
epoch =  {350} cost =  {5.58327431916317}
epoch =  {400} cost =  {5.349562803741289}
epoch =  {450} cost =  {5.133298143457496}
epoch =  {500} cost =  {5.032108134365417}
epoch =  {550} cost =  {4.859576871099943}
epoch =  {600} cost =  {4.777270604471077}
epoch =  {650} cost =  {4.71514349909412}


In [7]:
# Evaluation of Model on Dev Data
num_dev_batches = 90
num_batch = 256;
dev_dir = "/content/drive/MyDrive/Colab Notebooks/training_data/dev-test/"
with torch.no_grad():
   norm_error = 0
   for j in range(1, num_dev_batches+1):
      # Load test data from batch (i) into numpy here
      filename = dev_dir+"batch_"+ str(j).zfill(5) + ".hdf"
      f = h5py.File(filename, 'r')
      data = np.transpose(f['data'])
      Y_dev_np = data[:,0:2]
      X_dev_np = data[:,2:]
      X_dev = torch.from_numpy(X_dev_np)
      Y_dev = torch.from_numpy(Y_dev_np)
      Y_dev_prediction = Model(X_dev)
      norm_error += torch.norm(Y_dev-Y_dev_prediction)/(torch.norm(Y_dev))
      norm_error /= (num_dev_batches*num_batch)
print("Dev error: ", norm_error.item())

Dev error:  4.061973760410526e-06


In [8]:
# Evaluation of Model on Test Data
test_dir = "/content/drive/MyDrive/Colab Notebooks/training_data/dev-test/"
with torch.no_grad():
   norm_error = 0
   for j in range(91, 180):
      # Load test data from batch (i) into numpy here
      filename = test_dir+"batch_"+ str(j).zfill(5) + ".hdf"
      f = h5py.File(filename, 'r')
      data = np.transpose(f['data'])
      Y_test_np = data[:,0:2]
      X_test_np = data[:,2:]
      X_test = torch.from_numpy(X_test_np)
      Y_test = torch.from_numpy(Y_test_np)
      Y_test_prediction = Model(X_test)
      norm_error += torch.norm(Y_test-Y_test_prediction)/(torch.norm(Y_test))
      norm_error /= (num_dev_batches*num_batch)
print("Test error: ", norm_error.item())

Test error:  6.895311220939579e-06


In [6]:
# TO DO: Save model